<h1><center> H&M Personalized Fashion Recommendations</center></h1>
<center><img src="https://static.euronews.com/articles/stories/04/98/67/36/1440x810_cmsv2_4f7e599c-edce-5354-b734-d1e960153442-4986736.jpg" width="400px"><center>
<h3><center>Provide product recommendations based on previous purchases</center></h3>
    Develop product recommendations based on data from previous transactions, as well as from customer and product meta data.

#### Data Description

##### Files

**images/** - a folder of images corresponding to each article_id; images are placed in subfolders starting with the first three digits of the article_id; note, not all article_id values have a corresponding image.

**articles.csv** - detailed metadata for each article_id available for purchase

**customers.csv** - metadata for each customer_id in dataset

**sample_submission.csv** - a sample submission file in the correct format

**transactions_train.csv** - the training data, consisting of the purchases each customer for each date, as well as additional information. Duplicate rows correspond to multiple purchases of the same item. Your task is to predict the article_ids each customer will purchase during the 7-day period immediately after the training data period.

**NOTE:** You must make predictions for all customer_id values found in the sample submission. All customers who made purchases during the test period are scored, regardless of whether they had purchase history in the training data.

<h2><center> Diving Into DATA <center>

In [ ]:
## Utility Functions


def plot_percentage(ax, total): 
    for p in ax.patches:
        percentage = '{:.2f}%'.format(100*p.get_height()/total)
        x = p.get_x() + p.get_width() / 2 - 0.25
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size = 12)
    
def plot_distribution(dataframe, columns, ylabel):
    for column in columns:
        result= dataframe.groupby(column).size()
        plt.figure(figsize=(20,10))
        plt.xticks(rotation=90)
        plt.title("Distribution of "+column, fontsize=20)
        plt.ylabel(ylabel)
        ax = sns.barplot(x=result.index, y=result.values)
        total= dataframe.shape[0]
        plot_percentage(ax, total)
        

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
articles= pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
pd.set_option('display.max_colwidth', None)
articles.head()

In [ ]:
articles.nunique()
#Unique values for all the columns

In [ ]:
# Lets deep dive into Relevant columns to understand more about the characteristics of articles.

columns= ['product_group_name','index_name','index_group_name','garment_group_name']

plot_distribution(articles, columns,ylabel="Number of Articles")

#### OBSERVATIONS:
1) Based on product_group_name, H&M insuatry products are dominated by **Garment Upper Body** with **40.5%** of products, followd by **Garment Lower Body, Garment Full Body and Accessories** with **19.7 and 12.5% and 10.5%** respectively. All others products are 5% or less than that.

2) **Ladieswear and Babywear** segment is close to **70% together** and rest is the Menswear, Divided and Sports category.

3) Looking into the **Garment Groups**, we can see that the **large proportion** of the products are **Woollen wear(Jersey, Knitwear)** followed by Accessories.


In [ ]:
customers= pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

customers.head()

In [ ]:
columns= ['club_member_status','fashion_news_frequency','age']
plot_distribution(customers, columns,ylabel="Number of Customers")

#### OBSERVATIONS:

1) 92.75% of customers have their member status **Active** and very few members have actually left the club. But I think that does not mean that all the 92% customers are active in terms of purchases. Their account could be active but in dormant state.

2) There is an interesting insight from the **Age Distribution**. We can see that the maximum customers for H&M are from **Age Group 18-25** and then customer proportion starts decreasing which is obvious. But the **45-50+ age group** also has a good proportion and the demand increases for the products. 

#### We can further deep dive to understand the type of products that are prominent in both the age groups. 

#### Is there any differentiating factor in the product groups or H&M is targeting both the segments in the same product groups?? ---- We will see that further in the transactions data

In [ ]:
transactions_train= pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
transactions_train.head()

In [ ]:
transactions_train.shape

In [ ]:
# Join transaction data with customers and articles data to Understand the transactional behavior
merged= (transactions_train.merge(customers, on='customer_id', how='left'))
merged.head()


In [ ]:
merged2= (merged.merge(articles, on='article_id', how='left'))
merged2.head()

In [ ]:
## Since the trasactional data is quite big. Merging the customers and articles both dataframes is
## giving me error as RAM crashes. So the alternate solution:
## Reference :https://stackoverflow.com/questions/47386405/memoryerror-when-i-merge-two-pandas-data-frames

df1 = transactions_train
df2 = customers
df3= articles

# creating a empty bucket to save result
df_result = pd.DataFrame(columns=(df1.columns.append(df2.columns.append(df3.columns))).unique())

# deleting df to save memory
del(df1)

def preprocess(x,df_result):
    inter_res=(x.merge(df2, on = 'customer_id', how='left')).merge(df3,on='article_id', how='left')
    df_result=pd.concat([df_result,inter_res], axis=0)
    

reader = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', chunksize=1000) # chunksize depends with you colsize

for r in reader:
    df_result= preprocess(r,df_result)


In [ ]:
df_result.shape

In [ ]:
df_result.to_csv('../input/h-and-m-personalized-fashion-recommendations/merged_data.csv')

<center><h2> Understanding transactional behaviour </h2></center>

<CENTER><H2> WORK IN PROGRESS............  </H2></CENTER>